In [140]:
import pretty_midi as pm
chrom_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
offsets = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11,
    '8': 12,
    '9': 14,
    '10': 16,
    '11': 17,
    '12': 19,
    '13': 21
}
# Returns a note name based on its MIDI note number
def get_note(note_n):
    return chrom_notes[note_n % 12]

# Returns the correct note based on a root note of a scale and its degree
def parse_chord(root, number_string):
    note_num = chrom_notes.index(root)
    out = ""
    num = ""
    scale_num = 0
    parentheses = False
    for char in number_string:
        if char == '(':
            parentheses = True
        if char == 'b':
            scale_num -= 1
        if char == '#':
            scale_num += 1
        if char >= '0' and char <= '9':
            num += char
    scale_num += offsets.get(num)
    if (parentheses):
        out = "("
    out += str(get_note(note_num + scale_num))
    if (parentheses):
        out += ")"
    return out

In [141]:
# Actually outputs the chord list (dict version)
chord_list = []
for note in chrom_notes:
    f = open("../chords without names.txt")
    lines = f.readlines()
    for line in lines:
        parts = line.split()
        chord_name = ''
        note_list = []
        for i in range(len(parts)):
            part = parts[i]
            if i == 0:
                chord_name = part.replace('_', note, 1)
            elif part[0] == 'b' or part[0] == '#' or \
               (part[0] >= '0' and part[0] <= '9') or \
               part[0] == '(':
                note_list.append(parse_chord(note, part))
            else: continue
        chord_list.append([chord_name, note_list])

In [142]:
song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Untitled score.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\7-Strings Ensemble Staccato 7 test.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\jazz but without the piano chords.mid")
# song = pm.PrettyMIDI("C:\\Users\\Tim\\Downloads\\blind comp render E-PIANO ONLY_basic_pitch.mid")

In [143]:
song.instruments

[Instrument(program=52, is_drum=False, name="Men")]

In [144]:
for inst in song.instruments:
    print(inst.name)
    note_scores = []
    note_scores_octave_agn = []
    for i in range(0, 128):
        note_scores.append(0)
    for i in range(0, 12):
        note_scores_octave_agn.append(0)
    for note in inst.notes:
        duration = note.end - note.start
        score = duration * note.velocity / 127
        # octave_multiplier = max(0, 1 - (max(0, (round(note.pitch % 12) - 3) / 8.0)))
        # print("%d %f" % (note.pitch, octave_multiplier))
        octave_multiplier = 1
        score *= octave_multiplier
        note_scores[note.pitch] += score
        note_scores_octave_agn[note.pitch % 12] += score
    note_scores_dict = dict()
    note_scores_octave_agn_dict = dict()
    for i in range(0, 128):
        if note_scores[i] != 0:
            note_scores_dict[i] = note_scores[i]
    for i in range(0, 12):
        if note_scores_octave_agn[i] != 0:
            note_scores_octave_agn_dict[i] = note_scores_octave_agn[i]
    note_scores_sorted = sorted(note_scores_dict.items(), key=lambda x:x[1], reverse = True)
    note_scores_dict = dict(note_scores_sorted)
    note_scores_octave_agn_sorted = sorted(note_scores_octave_agn_dict.items(), key=lambda x:x[1], reverse = True)
    for key in note_scores_octave_agn_dict.keys():
        print("Note %s has a score of %f" % (get_note(key), note_scores_octave_agn_dict[key]))
    print("The most common note is %s" % get_note(note_scores_octave_agn_sorted[0][0]))
    print("------")
    print()

Men
Note C has a score of 1.259186
Note E has a score of 1.259186
Note F has a score of 1.259186
Note G has a score of 1.259186
Note Ab has a score of 1.259186
The most common note is C
------



In [145]:
chord_scores_dict = {}
for chord_tuple in chord_list:
    chord_name = chord_tuple[0]
    chord_notes = chord_tuple[1]
    chord_score = 0.0
    for note in chord_notes:
        multiplier = 1
        actual_note = note
        if note[0] == '(':
            multiplier = 0.5
            actual_note = note[1 : (len(note) - 1)]
        note_val = chrom_notes.index(actual_note)
        note_score = note_scores_octave_agn_dict.get(note_val, 0)
        if note_score <= 0.1:
            note_score = -0.3 # Deweight chords with missing notes
        chord_score += note_score
    if chord_score != 0.0:
        chord_scores_dict[chord_name] = chord_score
       
chord_scores_dict_sorted = sorted(chord_scores_dict.items(), key=lambda x:x[1], reverse = True)

print("The 20 highest-scoring chords are:")
for i in range(20):
    print("%d: %s with a score of %f" % ((i + 1), chord_scores_dict_sorted[i][0], chord_scores_dict_sorted[i][1]))

The 20 highest-scoring chords are:
1: Fm/M7 with a score of 6.295932
2: Bb7#11 with a score of 5.695932
3: AbM7b6 with a score of 5.395932
4: CM7b6 with a score of 5.395932
5: Csus4 with a score of 5.036745
6: Fsus2 with a score of 5.036745
7: AbaugM7 with a score of 5.036745
8: C7sus4 with a score of 4.736745
9: FM9 with a score of 4.736745
10: Fm9 with a score of 4.736745
11: G7sus4 with a score of 4.736745
12: Cb9sus with a score of 4.436745
13: Dm11 with a score of 4.436745
14: Ebb9sus with a score of 4.436745
15: F7#9 with a score of 4.436745
16: Fm11 with a score of 4.436745
17: Gb9sus with a score of 4.436745
18: Bb13 with a score of 4.436745
19: CM13 with a score of 4.136745
20: DbM#4 with a score of 4.136745
